# PDF to CSV

**Note**

[令和６年 (2024-03-01) の PDF](https://www.kyoukaikenpo.or.jp/~/media/Files/shared/hokenryouritu/r6/ippan/r60213tokyo.pdf)
は、直接 tabula-py でパース出来ないため、次の手順でパース出来る PDF に変換する。

- [都道府県別エクセル](https://www.kyoukaikenpo.or.jp/~/media/Files/shared/hokenryouritu/r6/ippan/r6ippan3.xlsx)をダウンロード
- ファイルをエクセルで開き PDF で保存 (ファイル > 印刷)
- PDF を所定の場所に配置 (`/path/to/r6/kyoukaikenpo.or.jp/`)
- ローカルサーバーを 8000 番ポートで起動 (`cd /path/to/ && python -m http.server`)
- このノートブックを実行することで CSV に変換

In [1]:
# SRC = 'https://www.kyoukaikenpo.or.jp/~/media/Files/shared/hokenryouritu/r6/ippan/r60213tokyo.pdf'
SRC = 'http://localhost:8000/r6/kyoukaikenpo.or.jp/r60213tokyo.pdf'
DST = '2024-03-01.csv'

In [2]:
import datetime
import hashlib
import io
import pathlib
import urllib.request

try:
    import tabula
except ModuleNotFoundError:
    !pip install tabula-py==2.9.0
    import tabula

datetime.datetime.now().astimezone(datetime.timezone.utc).isoformat()

'2024-03-03T11:17:50.777603+00:00'

In [3]:
data = io.BytesIO()

with urllib.request.urlopen(SRC) as f:
    data.write(f.read())

hashlib.sha256(data.getvalue()).hexdigest()

'f8041ecc055362dd932ce6a1ac1867c1289426be47a4059f167e39d7a05b8458'

In [4]:
data.seek(0)

df = tabula.read_pdf(data, lattice=True, pages=1)[0]

df = df.iloc[:, [1, 2, 3, 4, 5, 6, 7]]

df.columns = """
標準報酬_等級
標準報酬_月額
報酬月額
介護保険の被保険者でない場合_全額
介護保険の被保険者でない場合_折半額
介護保険の被保険者の場合_全額
介護保険の被保険者の場合_折半額
""".strip().split()

df['標準報酬_等級'] = df['標準報酬_等級'].str.replace(r'\([0-9]+\)', '', regex=True)
df = df[~df['標準報酬_等級'].isnull()]
df = df[df['標準報酬_等級'].str.match(r'^[0-9]+$')]
df['標準報酬_等級'] = df['標準報酬_等級'].astype(int)
df = df.set_index('標準報酬_等級').sort_index()

assert df.shape[0] == 50, f'等級数 ({df.shape[0]}) に過不足があります。'

s = df['報酬月額'].str.replace(r'[^0-9~]', '', regex=True)
df_ = s.str.split('~', n=1, expand=True)
df_ = df_.rename(columns={0: '報酬月額_以上', 1: '報酬月額_未満'})
df = df.merge(df_, left_index=True, right_index=True)

del df['報酬月額']

for x in df.columns:
    df[x] = df[x].str.replace(',', '')

df = df.replace('', float('nan'))
df['報酬月額_以上'] = df['報酬月額_以上'].astype(float)
df['報酬月額_未満'] = df['報酬月額_未満'].astype(float)
df['標準報酬_月額'] = df['標準報酬_月額'].astype(int)

for x in """
介護保険の被保険者でない場合_全額
介護保険の被保険者でない場合_折半額
介護保険の被保険者の場合_全額
介護保険の被保険者の場合_折半額
""".strip().split():
    df[x] = df[x].astype(float)

for x in """
介護保険の被保険者でない場合_
介護保険の被保険者の場合_
""".strip().split():
    df[x + '控除額'] = (df[x + '折半額'] + .49).astype(int)

df = df["""
標準報酬_月額
報酬月額_以上
報酬月額_未満
介護保険の被保険者でない場合_全額
介護保険の被保険者でない場合_折半額
介護保険の被保険者でない場合_控除額
介護保険の被保険者の場合_全額
介護保険の被保険者の場合_折半額
介護保険の被保険者の場合_控除額
""".strip().split()]

df.to_csv(DST)

hashlib.sha256(pathlib.Path(DST).read_bytes()).hexdigest()

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Got stderr: 3月 03, 2024 8:17:51 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



'035311e5823cc8b5b562c6263fdfbaf51d8e398c7f6fe7deee73aabde4345f25'